In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install torch
!pip install python-dotenv
!pip install pysubs2
!pip install pandas
!pip install srt
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import Trainer
from transformers import TrainerCallback
import os
import srt
import pysubs2
import pandas as pd
import time
from datasets import load_dataset, Dataset, concatenate_datasets, load_dataset_builder
from dotenv import load_dotenv
from accelerate import Accelerator
from google.colab import userdata
from sklearn.model_selection import train_test_split

In [ ]:
# Load the API token from the .env file
api_token = userdata.get('HF_TOKEN')  # or use os.environ.get

# Define the model ID
model_name = "openai-community/gpt2-large"

# Clear any existing cache
torch.cuda.empty_cache()

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=api_token)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=api_token,
    torch_dtype=torch.bfloat16,
)

# Resize token embeddings
model.resize_token_embeddings(len(tokenizer))

# Verify if the model is on the correct device
print(f"Model is on device: {next(model.parameters()).device}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model is on device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def parse_srt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        subtitles = list(srt.parse(content))
        return [subtitle.content for subtitle in subtitles]

def parse_ass(file_path):
    subs = pysubs2.load(file_path)
    return [event.text for event in subs.events if event.type == "Dialogue"]

def extract_dialogues(directory):
    dialogues = []
    for filename in os.listdir(directory):
        if filename.endswith(".srt"):
            dialogues.extend(parse_srt(os.path.join(directory, filename)))
        elif filename.endswith(".ass"):
            dialogues.extend(parse_ass(os.path.join(directory, filename)))
    return dialogues

# Directory containing subtitle files
subtitle_directory = '/content/drive/My Drive/animegirlCS/anime scripts'
# Extract dialogues
anime_scripts = extract_dialogues(subtitle_directory)

# Prepare dataset from scripts
df = pd.DataFrame({"text": anime_scripts})
anime_scripts_dataset = Dataset.from_pandas(df)
print(anime_scripts_dataset)

Dataset({
    features: ['text'],
    num_rows: 32059
})


In [ ]:
# Load the dataset builder for the Python subset
builder = load_dataset_builder("code_search_net", "python", trust_remote_code=True)

# Download and prepare the dataset
builder.download_and_prepare()

# Load the dataset from the builder
dataset = builder.as_dataset(split="train")

# Extract the relevant text data (e.g., function documentation and code)
def extract_code_search_net_data(example):
    return {"text": example["whole_func_string"]}

code_search_net_texts = dataset.map(extract_code_search_net_data, batched=True, remove_columns=dataset.column_names)
print("Extracted text from CodeSearchNet dataset")

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

Extracted text from CodeSearchNet dataset


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)
# Assuming your anime scripts are structured for QA
tokenized_anime_dataset = anime_scripts_dataset.map(tokenize_function, batched=True)
tokenized_code_search_net_dataset = code_search_net_texts.map(tokenize_function, batched=True)

# Combine the datasets
combined_dataset = concatenate_datasets([tokenized_anime_dataset, tokenized_code_search_net_dataset])
combined_dataset = combined_dataset.remove_columns(["text"])  # Remove the original text column
#def add_labels(examples):
#    examples["labels"] = examples["input_ids"].copy()
#    return examples

# Apply the function to the dataset
#combined = combined_dataset.map(add_labels, batched=True)

Map:   0%|          | 0/32059 [00:00<?, ? examples/s]

Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

In [ ]:
print(f"Combined dataset: {combined_dataset}")

Combined dataset: Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 444237
})


In [ ]:
#def check_dataset_for_labels(dataset):
#    sample = dataset[0]
#    if 'labels' not in sample:
#        raise ValueError("Dataset does not contain 'labels' key.")
#    else:
#        print("Sample labels:", sample['labels'])
#
# Verify dataset
#check_dataset_for_labels(combined)

Sample labels: [101, 11320, 2474, 11320, 2474, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
from torch.utils.data import Dataset
train_index, val_index = train_test_split(torch.arange(len(combined_dataset)), test_size=0.2)
train_data = combined_dataset[train_index]
val_data = combined_dataset[val_index]

class CustomDataset(Dataset):
    def __init__(self, data):
        self.input_ids = data['input_ids']
        self.attention_mask = data['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
        }
        return item

train_dataset = CustomDataset(train_data)
val_dataset = CustomDataset(val_data)

In [ ]:
print(train_dataset[0])

{'input_ids': tensor([ 4299,   318,    62,   439,    62,  3245,    62, 23108,     7,   944,
         2599,   198,   220,   220,   220,   220,   220,   220,   220, 37227,
          198,   220,   220,   220,   220,   220,   220,   220,  1058,    81,
         4906,    25, 20512,   198,   220,   220,   220,   220,   220,   220,
          220, 37227,   628,   220,   220,   220,   220,   220,   220,   220,
          611,  2116, 13557,   312,    62,   318,   407,  6045,    25,   198,
          220,   220,   220,   220,   220,   220,   220,   220,   220,   220,
          220,  1441, 10352,   628,   220,   220,   220,   220,   220,   220,
          220,   611,  2116, 13557, 25598,   318,   407,  6045,    25,   198,
          220,   220,   220,   220,   220,   220,   220,   220,   220,   220,
          220,  1441, 10352,   628,   220,   220,   220,   220,   220,   220,
          220,   611,  2116, 13557, 43162,   318,   407,  6045,    25,   198,
          220,   220,   220,   220,   220,   220, 

In [ ]:
from transformers import DataCollatorForLanguageModeling
training_args = TrainingArguments(
    output_dir="./results",                   # Directory to save the model checkpoints
    evaluation_strategy="epoch",              # Evaluate after every epoch
    learning_rate=2e-5,                       # Learning rate
    per_device_train_batch_size=16,           # Batch size for training
    per_device_eval_batch_size=16,            # Batch size for evaluation
    num_train_epochs=1,                       # Number of training epochs
    weight_decay=0.01,                        # Weight decay
    logging_steps=500,                        # Log every 500 steps
    save_strategy='epoch',                    # Save after training
    save_total_limit=2,                       # Limit the total amount of checkpoints to keep
    remove_unused_columns=False,              # Do not remove unused columns
    load_best_model_at_end=True,              # Load the best model at the end of training
    metric_for_best_model="loss",             # Use the loss to determine the best model
    greater_is_better=False,                  # Set to True if you want to minimize the metric
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()
drive_path1 = "/content/drive/My Drive/fine-tuned-GPT2"

# Save the fine-tuned model and tokenizer with descriptive names
model.save_pretrained(drive_path1 + "/model")
tokenizer.save_pretrained(drive_path1 + "/tokenizer")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.662400,1.608710


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


('/content/drive/My Drive/fine-tuned-GPT2/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/fine-tuned-GPT2/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/fine-tuned-GPT2/tokenizer/vocab.json',
 '/content/drive/My Drive/fine-tuned-GPT2/tokenizer/merges.txt',
 '/content/drive/My Drive/fine-tuned-GPT2/tokenizer/added_tokens.json',
 '/content/drive/My Drive/fine-tuned-GPT2/tokenizer/tokenizer.json')

In [ ]:
print(model.state_dict().keys())

odict_keys(['transformer.wte.weight', 'transformer.wpe.weight', 'transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.1.ln_1.weight', 'transformer.h.1.ln_1.bias', 'transformer.h.1.attn.c_attn.weight', 'transformer.h.1.attn.c_attn.bias', 'transformer.h.1.attn.c_proj.weight', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.ln_2.weight', 'transformer.h.1.ln_2.bias', 'transformer.h.1.mlp.c_fc.weight', 'transformer.h.1.mlp.c_fc.bias', 'transformer.h.1.mlp.c_proj.weight', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.2.ln_1.weight', 'transformer.h.2.ln_1.bias', 'transformer.h.2.attn.c_attn.weight', 'transformer.h.2.attn.